In [52]:
!uv pip install transformers matplotlib pandas seaborn torch

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from huggingface_hub import hf_hub_download
from difflib import SequenceMatcher
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

Audited 5 packages in 28ms


#### utils


In [ ]:
def get_random_needles(df_needles, n=5):
    """
    Get random needles of different types from the dataframe.
    """
    df = df_needles.sample(n=n)
    df = df.drop_duplicates(subset=["arg1"], keep="first").reset_index(drop=True)
    return df


def generate_context(df_haystack, df_needles) -> str:
    """
    Generate context for the promp.
    """
    context = [" "]
    for i in range(len(df_needles)):
        needle = df_needles.iloc[i]["needle"]
        haystack = ""
        if i < len(df_needles) - 1:
            haystack = df_haystack.iloc[i]["text"]

        context.append(needle)
        if haystack:
            context.append(" " + haystack + " ")
    return "".join(context)


def generate_messages(df_needles, df_haystack, n=5):
    """
    Generate messages for the model.
    Args:
        df_needles (pd.DataFrame): DataFrame containing the needles.
        df_haystack (pd.DataFrame): DataFrame containing the haystacks.
        n (int): Number of random needles to select.
    Returns:
        messages (list): List of messages for the model.
        prompt_needle (pd.Series): Random needle selected for the prompt.
    """
    df_rand_needles = get_random_needles(df_needles, n=n)
    context = generate_context(df_haystack, df_rand_needles)

    prompt_needle = df_rand_needles.sample(n=1).iloc[0]

    sys_prompt = "You are an intelligent AI assistant skilled in answering user questions base on documents provided by the user. Please keep your answers concise and clear. Do not talk about irrelevant topics or repeat your answers. The document given to you by the user is:"
    messages = [
        {
            "role": "system",
            "content": sys_prompt,
        },
        {
            "role": "user",
            "content": context,
        },
        {
            "role": "user",
            "content": prompt_needle["retrieval_question"],
        },
    ]
    return messages, prompt_needle


def grade(response, answer) -> float:
    return float(SequenceMatcher(None, response, answer).ratio())


def find_subtensor_indices(
    haystack: torch.LongTensor, needle: torch.LongTensor
) -> torch.LongTensor:
    """
    Returns a 1D tensor of all start‐positions where `needle`
    appears as a contiguous slice of `haystack`.
    """
    n, m = haystack.size(0), needle.size(0)
    if m == 0:
        # every position (including “after” the last) is a match
        return torch.arange(n + 1, dtype=torch.long)
    if m > n:
        return torch.empty(0, dtype=torch.long)

    # create all length‐m windows: shape (n-m+1, m)
    windows = haystack.unfold(0, m, 1)  # → (n-m+1)×m
    # compare each window to needle, then all dims must match
    matches = (windows == needle).all(dim=1)  # → (n-m+1)
    # extract the indices where True
    return matches.nonzero(as_tuple=True)[0]


def block_reduce(
    matrix: torch.Tensor,
    block_size: int = 64,
    mode: str = "max",  # one of "max" or "mean"
) -> torch.Tensor:
    """
    Collapse each contiguous block of `block_size` columns in every row of `matrix`
    down to either its maximum or its average, returning a tensor of shape
    (a, ceil(b/block_size)).

    Parameters
    ----------
    matrix : torch.Tensor
        2D tensor of shape (a, b).
    block_size : int
        Number of columns per block (default 64).
    mode : str
        Reduction to apply: "max" or "mean".

    Returns
    -------
    torch.Tensor
        2D tensor of shape (a, num_blocks) where num_blocks = ceil(b / block_size),
        each entry is the max or mean over that block in the original row.
    """
    if matrix.dim() != 2:
        raise ValueError("`matrix` must be 2-dimensional")
    if mode not in {"max", "mean"}:
        raise ValueError("mode must be 'max' or 'mean'")

    a, b = matrix.shape
    full_blocks = b // block_size

    # handle all full blocks
    if full_blocks > 0:
        blocks = matrix[:, : full_blocks * block_size].unfold(
            1, block_size, block_size
        )  # → (a, full_blocks, block_size)
        if mode == "max":
            full_reduced, _ = blocks.max(dim=2)  # → (a, full_blocks)
        else:  # mean
            full_reduced = blocks.mean(dim=2)  # → (a, full_blocks)
    else:
        full_reduced = matrix.new_empty((a, 0))

    # handle any remainder
    rem = b - full_blocks * block_size
    if rem > 0:
        tail = matrix[:, full_blocks * block_size :]  # → (a, rem)
        if mode == "max":
            rem_reduced, _ = tail.max(dim=1, keepdim=True)  # → (a,1)
        else:
            rem_reduced = tail.mean(dim=1, keepdim=True)  # → (a,1)
        return torch.cat([full_reduced, rem_reduced], dim=1)

    return full_reduced


#### dataset


In [ ]:
df_needles = pd.read_parquet(
    hf_hub_download(
        repo_id="opencompass/NeedleBench",
        filename="retrieval_needles/test/0000.parquet",
        repo_type="dataset",
        revision="refs/convert/parquet",
    )
)
df_needles = df_needles[df_needles["language"] == "English"].reset_index(drop=True)

df_haystack = pd.read_parquet(
    hf_hub_download(
        repo_id="opencompass/NeedleBench",
        filename="en_haystack_texts/test/0000.parquet",
        repo_type="dataset",
        revision="refs/convert/parquet",
    )
)

df_haystack = df_haystack[
    df_haystack["text"].str.len().between(5000, 7500)
].reset_index(drop=True)

0


#### model


In [ ]:
MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"
# "Qwen/Qwen2.5-0.5B-Instruct"

MAX_LENGTH = 512

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto",
    output_attentions=True,
    attn_implementation="eager",
    return_dict_in_generate=True,
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
messages, prompt_needle = generate_messages(df_needles, df_haystack, n=4)

with torch.no_grad():
    torch.cuda.empty_cache()

text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

inputs = tokenizer([text], return_tensors="pt").to(model.device)
if len(inputs.input_ids[0]) <= 5000:
    with torch.inference_mode():
        output = model.generate(
            **inputs,
            max_new_tokens=MAX_LENGTH,
            output_attentions=True,
            return_dict_in_generate=True,
            use_cache=True,
            do_sample=True,
        )

    generated_ids = [
        output_ids[len(input_ids) :]
        for input_ids, output_ids in zip(inputs.input_ids, output.sequences)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
else:
    print(f"Text length: {len(text)}")
    print(f"Context length ({len(inputs.input_ids[0])} tokens) too long")
    raise Exception(
        f"Text length: {len(text)}\nContext length ({len(inputs.input_ids[0])} tokens) too long"
    )

In [ ]:
print(f"Text Length: {len(text)}")
print(f"Context Length: {len(inputs.input_ids[0])}")
print(f"Question: {prompt_needle['retrieval_question']}")
print(f"Correct Answer: {prompt_needle['gold_standard_answer']}")
print(f"Response: {response}")
print(f"Grade: {grade(response, prompt_needle['gold_standard_answer'])}")
print(f"Needle Position: {prompt_needle.name}")

with open("log.txt", "w") as f:
    f.write(f"Text Length: {len(text)}\n")
    f.write(f"Context Length: {len(inputs.input_ids[0])}\n")
    f.write(f"Question: {prompt_needle['retrieval_question']}\n")
    f.write(f"Correct Answer: {prompt_needle['gold_standard_answer']}\n")
    f.write(f"Response: {response}\n")
    f.write(f"Grade: {grade(response, prompt_needle['gold_standard_answer'])}\n")
    f.write(f"Needle Position: {prompt_needle.name}")


#### heatmaps


In [ ]:
random_string_tokens = tokenizer(
    [" " + prompt_needle["needle"]], return_tensors="pt"
).to(model.device)
random_string_tokens = random_string_tokens.input_ids[0]

indices = find_subtensor_indices(inputs.input_ids[0], random_string_tokens)

index_start = indices[0].item()
index_end = index_start + len(random_string_tokens)

n_layers = 28
n_heads = 12
div_factor = 4

attention = output.attentions
for layer in range(n_layers):
    fig, axes = plt.subplots(
        div_factor, n_heads // div_factor, figsize=(8 * div_factor, 5 * div_factor)
    )
    for i, ax in enumerate(axes.flat):
        weights = attention[0][layer][0][i][index_start:index_end]
        weights = block_reduce(weights, mode="mean").cpu().float().numpy()
        sns.heatmap(
            weights,
            ax=ax,
            cmap="bone",
            yticklabels=False,
        )
        ax.set_title(f"Head {i + 1}")
    plt.tight_layout()
    plt.savefig(f"imgs_mean/layer_{layer + 1:02}_mean.png")
    plt.close()

for layer in range(n_layers):
    fig, axes = plt.subplots(2, n_heads // 2, figsize=(50, 10))
    for i, ax in enumerate(axes.flat):
        weights = attention[0][layer][0][i][index_start:index_end]
        weights = block_reduce(weights).cpu().float().numpy()
        sns.heatmap(
            weights,
            ax=ax,
            cmap="bone",
            yticklabels=False,
        )
        ax.set_title(f"Head {i + 1}")
    plt.tight_layout()
    plt.savefig(f"imgs_max/layer_{layer + 1:02}_max.png")
    plt.close()

In [ ]:
!zip -r /content/imgs.zip /content/imgs_max/ /content/imgs_mean/ /content/log.txt

from google.colab import files

files.download("/content/imgs.zip")